In [1]:
pwd

'/home/wsuser/work'

In [3]:
!pip install keras==2.7.0


     |████████████████████████████████| 1.3 MB 14.1 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [31]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./225, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./225)

In [21]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='kxTodcnZnZV9ertCzD2UKE39UZrk_EI4_wFgzHbArMyl',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-zgcmaebuc27a5x'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']


# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [23]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [24]:
pwd

'/home/wsuser/work'

In [25]:
import os

In [26]:
filenames=os.listdir('/home/wsuser/work/Dataset/training_set')

In [32]:
x_train = train_datagen.flow_from_directory(r"E:\Development\Dataset\training_set",target_size=(64,64), batch_size=300,class_mode='categorical', color_mode ="grayscale")

Found 15750 images belonging to 9 classes.


In [33]:
x_test = test_datagen.flow_from_directory(r"E:\Development\Dataset\test_set",target_size=(64,64), batch_size=300,class_mode='categorical', color_mode ="grayscale")

Found 2250 images belonging to 9 classes.


In [34]:
x_train.class_indices
x_test.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

In [86]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten

In [87]:
model=Sequential()

In [88]:
model.add(Convolution2D(32,(3,3), input_shape=(64,64,3), activation = 'relu'))

In [89]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [90]:
model.add(Flatten())

In [91]:
model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=9,activation='softmax'))


In [92]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit_generator(x_train, steps_per_epoch=24, epochs=10,validation_data=x_test,validation_steps=40)

In [94]:
model.save('mainmodel.h5')

In [95]:
!tar -zcvf image-classification.model_new.tgz Balaji.h5

a Balaji.h5


In [96]:
ls -1

 Volume in drive C has no label.
 Volume Serial Number is 1E86-37BA

 Directory of C:\Users\BALAJI POWER MART



File Not Found


In [97]:
!pip install watson-machine-learning-client --upgrade

In [98]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url" : "https://us-south.ml.cloud.ibm.com",
                    "apikey": "p9Mb_jnWsy9gh1lLtn_EKsqtPI-lJFotjMJKUJ6fUpsJ"
                  }

client = APIClient(wml_credentials)

In [99]:
client = APIClient(wml_credentials)

In [48]:
def guid_from_space_name(client, space_name):
    space=client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name) ['metadata']['id'])

In [49]:
space_uid = guid_from_space_name(client, 'imageClassification')
print(space_uid)

f28d7902-3aa3-4577-b5cc-8913de60b4fa


In [50]:
client.set.default_space(space_uid)

'SUCCESS'

In [6]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [100]:
from keras.models import load_model
from keras.preprocessing import image


In [101]:
model= load_model('mainmodel.h5')

In [102]:
software_spec_uid=client.software_specifications.get_uid_by_name('tensorflow_2.4-py3.8-horovod')

software_spec_uid

'217c16f6-178f-56bf-824a-b19f20564c49'

In [103]:
img = image.load_img('1.png')

In [104]:
import numpy as np
x= image.img_to_array(img)
x= np.expand_dims(x,axis=0)

In [105]:
x.shape

(1, 64, 64, 3)

In [106]:
print(x)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [107]:
pred = np.argmax(model.predict(x),axis=1)
pred[0]

8

In [108]:
index = ['A','B','C','D','E','F','G','H','I','J']

In [109]:
a=index[pred[0]]

In [110]:
a

'I'